## GAN의 정의 및 구조

- 위조지폐범(생성자)와 경찰(구분자)로 비유하면, 위조지폐범은 경찰을 최대한 속이려 하고, 경찰은 위조한 지폐를 최대한 감별하려 한다는 것
- 이렇게 위조지폐를 만들고 감별하려는 경쟁을 하게 되면, 서로의 능력이 발전하게 되고 나중에는 위조지폐범은 진짜와 거의 구분할 수 없을 정도로 진짜 같은 위조지폐를 만들게 된다는 것

![GAN구조](./GAN.png)

1. 실제 이미지를 주고 구분자(Discriminator)에게 이미지가 진짜임을 판단하게 함
2. 생성자(Generator)를 통해 노이즈로부터 임의의 이미지를 만든다.
3. 만들어진 이미지를 다시 구분자를 통해 진짜인지를 판단하게 함
<hr>
이 과정을 통해 생성자를 구분자를 속여 진짜처럼 보이게 하고, 구분자는 생성자를 만든 이미지를 최대한 가짜라고 구분하도록 훈련

## GAN 기본 모델 구현

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# hyperparameter 설정
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128

In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise]) # 가짜 이미지 생성 용

생성자 신경망용

In [4]:
# 입력층
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
# 출력층(입력층의 크기와 같아야함)
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

구분자 신경망용

In [5]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

생성자 신경망 구성

In [6]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

구분자 신경망 구성

In [7]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

무작위 노이즈 생성 함수

In [16]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [9]:
G = generator(Z)
D_gene = discriminator(G) # 생성된 이미지 구분
D_real = discriminator(X) # 실제 이미지

#### 손실값
1. 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값(경찰 학습용)
  - 진짜 이미지 판별값(D_real)은 1에 가까워야 하고(진짜라 판별), 가짜 이미지 판별값(D_gene)은 0에 가까워야 함(가짜라 판별)
2. 진짜라고 판단하도록 하는 손실값(위조지폐범 학습용)
  - 가짜 이미지 판별값(D_gene)를 1에 가깝에 만들기만 하면 됨

경찰 학습용 손실값

In [10]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene)) # D_real과 (1-D_gene) 값을 더한 값

위조지폐범 학습용 손실값

In [11]:
loss_G = tf.reduce_mean(tf.log(D_gene)) # 해당 값을 1에 가깝게 만들어야 함

In [12]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [13]:
# 변수 최적화 함수 구성(loss값을 최대화 해야함, 음수 부호를 붙여줌)
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [21]:
# 학습 반복
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
    print('Epoch : %04d' %epoch, 'D loss : {:.4}'.format(loss_val_D), 'G loss : {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print('최적화 완료')

Epoch : 0000 D loss : -0.5673 G loss : -2.36
Epoch : 0001 D loss : -0.4801 G loss : -2.08
Epoch : 0002 D loss : -0.7785 G loss : -1.677
Epoch : 0003 D loss : -0.751 G loss : -1.733
Epoch : 0004 D loss : -0.7019 G loss : -1.786
Epoch : 0005 D loss : -0.6267 G loss : -2.071
Epoch : 0006 D loss : -0.7218 G loss : -1.987
Epoch : 0007 D loss : -0.5723 G loss : -2.049
Epoch : 0008 D loss : -0.835 G loss : -2.045
Epoch : 0009 D loss : -0.5594 G loss : -2.077
Epoch : 0010 D loss : -0.8304 G loss : -1.925
Epoch : 0011 D loss : -0.8206 G loss : -1.84
Epoch : 0012 D loss : -0.7875 G loss : -1.876
Epoch : 0013 D loss : -0.7518 G loss : -1.705
Epoch : 0014 D loss : -0.7852 G loss : -1.746
Epoch : 0015 D loss : -0.5862 G loss : -1.908
Epoch : 0016 D loss : -0.7883 G loss : -1.944
Epoch : 0017 D loss : -0.7935 G loss : -1.963
Epoch : 0018 D loss : -0.7496 G loss : -1.883
Epoch : 0019 D loss : -0.6518 G loss : -1.862
Epoch : 0020 D loss : -0.7265 G loss : -1.765
Epoch : 0021 D loss : -1.037 G loss : -

## 원하는 숫자 생성하기

In [2]:
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class]) # 노이즈와 실제 이미지 각각에 해당하는 숫자를 힌트로 넣어주는 용도
Z = tf.placeholder(tf.float32, [None, n_noise])

생성자 신경망 구성(use tf.layers)

In [3]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
    return output

구분자 신경망 구성(use tf.layers)

In [4]:
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation=None)
    return output

노이즈 생성 함수(균등 분포)

In [5]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [6]:
G = generator(Z, Y)
D_real = discriminator(X,Y)
D_gene = discriminator(G,Y,True)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


손실함수(구분자, 생성자)

In [7]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene # 이 값을 최소화(D_real은 1에 가까워야 하고, D_gene는 0에 가까워야 함)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

학습모델 구성(tf.get_collection 사용)

In [9]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})
        
    print('Epoch : %04d' %epoch, 'D loss : {:.4}'.format(loss_val_D), 'G loss : {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})
        
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28,28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print('최적화 완료')

Epoch : 0000 D loss : 0.01822 G loss : 7.69
Epoch : 0001 D loss : 0.003534 G loss : 10.29
Epoch : 0002 D loss : 0.003204 G loss : 10.84
Epoch : 0003 D loss : 0.003842 G loss : 8.779
Epoch : 0004 D loss : 0.002424 G loss : 9.634
Epoch : 0005 D loss : 0.004695 G loss : 10.2
Epoch : 0006 D loss : 0.002279 G loss : 10.92
Epoch : 0007 D loss : 0.005077 G loss : 8.932
Epoch : 0008 D loss : 0.008255 G loss : 8.729
Epoch : 0009 D loss : 0.01844 G loss : 7.132
Epoch : 0010 D loss : 0.04805 G loss : 6.635
Epoch : 0011 D loss : 0.09691 G loss : 6.728
Epoch : 0012 D loss : 0.106 G loss : 6.032
Epoch : 0013 D loss : 0.1022 G loss : 5.218
Epoch : 0014 D loss : 0.1816 G loss : 4.723
Epoch : 0015 D loss : 0.3645 G loss : 4.902
Epoch : 0016 D loss : 0.2593 G loss : 4.93
Epoch : 0017 D loss : 0.4327 G loss : 3.614
Epoch : 0018 D loss : 0.428 G loss : 3.459
Epoch : 0019 D loss : 0.5278 G loss : 4.15
Epoch : 0020 D loss : 0.3549 G loss : 3.897
Epoch : 0021 D loss : 0.464 G loss : 4.013
Epoch : 0022 D loss

참고 : https://goo.gl/ZvSvtm